<a href="https://colab.research.google.com/github/Aradhyakapil/CodeGen-Agent-LangGraph/blob/main/codeGEN_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai requests
!pip install -U langchain langchain-community langchain-openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.7 MB/s eta 0:00:00


In [ ]:
import os
import time
import requests
from langchain.agents import Tool, AgentExecutor, initialize_agent
from langchain_community.chat_models import ChatOpenAI
from langchain import PromptTemplate
from langchain.agents.agent_types import AgentType
from google.colab import userdata
# ──────── API Keys ────────
api_key = userdata.get("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = api_key
JUDGE0_API_KEY = userdata.get("JUDGE0_API_KEY")

# ──────── Prompt Template ────────
prompt_template = PromptTemplate(
    input_variables=["language", "user_request"],
    template="""
You are an AI assistant generating code using GPT‑4o‑mini.

User wants code in {language}.
If the user does not specify a language (i.e., {language} is blank), default to Python.

Requirements:
{user_request}

Provide a complete code snippet without explanation.
Only return the entire detailed code as plain text.
STRICTLY Do not include backticks, NEVER include ```python in response
STRICTLY Do not include explanations.
"""
)

# ──────── LLM ────────
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.3)

# ──────── Tool 1: Code Generator ────────
def generate_code(input_str: str) -> str:
    language, _, request = input_str.partition("|")
    prompt = prompt_template.format(language=language.strip(), user_request=request.strip())
    return llm.invoke(prompt).content

#Wraps the func into a LangChain tool that agent can use.
code_gen_tool = Tool(
    name="CodeGenerator",
    func=generate_code,
    description="Generate code from a requirement. Input: 'language|user requirement'"
)

# ──────── Tool 2: Judge0 Executor ────────
LANGUAGE_IDS = {"python": 71, "javascript": 63, "java": 62, "c++": 54, "c": 50}

def run_code(input_str: str) -> str:
    language, _, code = input_str.partition("|")
    lang = language.lower().strip()
    if lang not in LANGUAGE_IDS:
        return f"Unsupported language: {lang}"

    headers = {
        "content-type": "application/json",
        "X-RapidAPI-Key": JUDGE0_API_KEY,
        "X-RapidAPI-Host": "judge0-ce.p.rapidapi.com"
    }

    payload = {
        "source_code": code,
        "language_id": LANGUAGE_IDS[lang],
        "cpu_time_limit": 2,
        "memory_limit": 128000,
        "wait": False
    }

    resp = requests.post("https://judge0-ce.p.rapidapi.com/submissions", headers=headers, json=payload)
    token = resp.json().get("token")
    if not token:
        return "Error: Token not received."

    for _ in range(10):
        result = requests.get(f"https://judge0-ce.p.rapidapi.com/submissions/{token}", headers=headers).json()
        status_id = result.get("status", {}).get("id", 0)
        if status_id in [1, 2]:  # Processing
            time.sleep(1)
            continue
        return f"Status: {result['status']['description']}\nSTDOUT:\n{result.get('stdout')}\nSTDERR:\n{result.get('stderr')}"

    return "Timeout waiting for Judge0 result."

judge0_tool = Tool(
    name="RunCode",
    func=run_code,
    description="Run code via Judge0. Input: 'language|code to run'"
)

# ──────── Agent Setup ────────
tools = [code_gen_tool, judge0_tool]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True
)

# ──────── Usage ────────
def run_task(description: str, language: str):
    print("🤖 Running agent...")
    input_str = f"{language}|{description}"
    response = agent.run(input_str)
    print("\n📦 Agent Final Output:\n", response)

# ──────── Entry Point ────────
if __name__ == "__main__":
    task = input("Enter the coding task: ")
    lang = input("Language (default = python): ").strip() or "python"
    run_task(task, lang)


Enter the coding task: Compute the total number of fruits if there are 4 mangoes and 2 apples.
Language (default = python): python
🤖 Running agent...


> Entering new AgentExecutor chain...


<ipython-input-4-2386945348>:108: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent.run(input_str)


To compute the total number of fruits, I need to add the number of mangoes and apples together. The requirement is straightforward, so I will generate the code to perform this calculation.

Action: CodeGenerator  
Action Input: python|Compute the total number of fruits if there are 4 mangoes and 2 apples.  

Observation: fruits_mangoes = 4
fruits_apples = 2
total_fruits = fruits_mangoes + fruits_apples
print("Total number of fruits:", total_fruits)
Thought:I have generated the code to compute the total number of fruits. Now, I will run this code to get the result.

Action: RunCode  
Action Input: python|fruits_mangoes = 4\nfruits_apples = 2\ntotal_fruits = fruits_mangoes + fruits_apples\nprint("Total number of fruits:", total_fruits)  

Observation: Status: Runtime Error (NZEC)
STDOUT:
None
STDERR:
  File "script.py", line 1
    fruits_mangoes = 4\nfruits_apples = 2\ntotal_fruits = fruits_mangoes + fruits_apples\nprint("Total number of fruits:", total_fruits)  
                        